# Chassis Level (Ride Height) Experiments
## Imports

In [1]:
from pathlib import Path
import os
import subprocess
from datetime import datetime
from typing import List
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment, Extern_Alphanumeric_Signal

### Set Folder and File Names

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/ride_height/"

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

# Generate new folder for experiments
timestamp = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
experiment_folder = os.path.join(experiment_folder, timestamp)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

## Create Necessary Experiments Files

In [ ]:
car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

ride_heights=["Lift","Mittel","Abgesenkt","Tief"]

print(f"Car model file path: {car_model_file_path}")

for ride_height in ride_heights:

    experiment_file_path = os.path.join(experiment_folder, "ride_height_"+ride_height+".json")
    experiment_name = f"Ride height - {ride_height}"
    experiment_description = "ride height signal discovery using 4 levels."
    signal_selection ="all"
    print(f"Experiment file path: {experiment_file_path}")
    !python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{experiment_file_path}' --experiment_name '{experiment_name}' --experiment_description '{experiment_description}' --signal_selection {signal_selection}



## [Optional]: Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/ride_height/2025-06-02_17_25_17"
ride_heights=["Lift","Mittel","Abgesenkt","Tief"]

from pathlib import Path
import os
import subprocess
from datetime import datetime

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

# Measurements

## Measurement "Lift"
Before executing the next Cell:

Set the ride height to "Lift".


In [ ]:
ride_height = "Lift"
experiment_path = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path "{experiment_path}" --activate_logging true --num_samples 5


## Measurement "Mittel" (Normal)
Before executing the next Cell:

Set the ride height to "Mittel".

In [ ]:
ride_height = "Mittel"
experiment_path = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path "{experiment_path}" --activate_logging true --num_samples 5

## Measurement "Abgesenkt" (Lowered)
Before executing the next Cell:

Set the ride height to "Abgesenkt".

In [ ]:
ride_height = "Abgesenkt"
experiment_path = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path "{experiment_path}" --activate_logging true --num_samples 5

## Measurement "Tief" (Low)
Before executing the next Cell:

Set the ride height to "Tief".

In [ ]:
ride_height = "Tief"
experiment_path = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path "{experiment_path}" --activate_logging true --num_samples 5

# Evaluation

## [Optional] Display Experiment Metadata

In [ ]:
# Display Experiment
for ride_height in ride_heights:
    experiment_file = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"


## Load Experiments

Load all relevant experiments needed for this analysis

In [17]:
experiment_files = {}
experiments = {}

for ride_height in ride_heights:
    experiment_files[ride_height] = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
    experiments[ride_height] = Experiment.load(experiment_files[ride_height])

## Add Ground truth
Add ground truth data for steering wheel angle. By experiment design these are fixed values, and are added manually in this case

In [ ]:
for ride_height in ride_heights:
    experiments[ride_height].external_alphanumeric_measurements.append(Extern_Alphanumeric_Signal( name = "Gear", id = 1, values = []))
    values:List[str] = []
    for value in experiments[ride_height].measurements[0].values:
        values.append(ride_height)
    experiments[ride_height].external_alphanumeric_measurements[0].name = "Gear"
    experiments[ride_height].external_alphanumeric_measurements[0].values = deepcopy(values)
    print(f"Experiment ride height {ride_height}: {len(experiments[ride_height].external_alphanumeric_measurements[0].values)} ground truth values")

## Concat all experiments to combined_experiment

In [19]:
combined_experiment = deepcopy(experiments[ride_heights[0]])
combined_experiment.name = f"Non constant singals for ride height experiment"
combined_experiment.description = f"Non constant signals for ride height experiment with {len(ride_heights)} different ride_heights."

for ride_height in ride_heights[1:]:
    combined_experiment.experiment_runtime_seconds += experiments[ride_height].experiment_runtime_seconds
    for measurement in experiments[ride_height].measurements:
        for x in combined_experiment.measurements:
            if x.serverid == measurement.serverid:
                if x.did == measurement.did:
                    x.values.extend(measurement.values)
                    break
    combined_experiment.external_alphanumeric_measurements[0].values.extend(experiments[ride_height].external_alphanumeric_measurements[0].values)


## Filter

Keep only non constant signals

In [ ]:
number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements before filtering", )
print("Applaying remove constant values filter")

combined_experiment.keep_non_constant_signals(combined_experiment)

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements after filtering", )

## Save combined experiment file

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"experiment_non_constant_signals.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"

## [OPTIONAL] Remove all previously found constant signals from individual experiments

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"combined_experiment.json")
combined_experiment = Experiment.load(combined_experiment_file)

for ride_height in ride_heights:
    experiments[ride_height].keep_signals_by_list(signals_list=combined_experiment.measurements)
    experiment_file = os.path.join(experiment_folder, f"ride_height_{ride_height}.json")
    experiments[ride_height].save(f"{experiment_file}")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"